# Building an ETL Pipeline

As the second part of the predict for Gather, you will need to build a pipeline of functions in python which does the following:

1. Function to connect to twitter and scrapes "Eskom_SA" tweets.
<br>
<br>
2. Cleans/Processes the tweets from the scraped tweets which will create a dataframe with two new columns using the following functions: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Hashtag Remover from Analyse Functions
<br>
<br>
3. Functions which connects to your SQL database and uploads the tweets into the table you store the tweets in the database.

In [1]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For numerical computation
import json
# For plotting and visualization:
from IPython.display import display
import pyodbc

# Consumer and Access details

Fill in your Consumer and Access details you should have recieved when applying for a Twitter API. 

In [2]:
# Consumer:
CONSUMER_KEY    = 
CONSUMER_SECRET = 

# Access:
ACCESS_TOKEN  = 
ACCESS_SECRET = 

In [0]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

# Function 1:

Write a function which:
- Scrapes _"Eskom_SA"_ tweets from Twitter. 

Function Specifications:
- The function should return a dataframe with the scraped tweets with just the "_Tweets_" and "_Date_". 
- Will take in the ```consumer key,  consumer secret code, access token``` and ```access secret code```.

NOTE:
The dataframe should have the same column names as those in your SQL Database table where you store the tweets.

In [18]:
def twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET):
    # Code Here
      
    # API's setup:
    def twitter_setup():
        """
        Utility function to setup the Twitter's API
        with access and consumer keys from Twitter.
        """
        # Authentication and access using keys:
        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

        # Return API with authentication:
        api = tweepy.API(auth, timeout=1000)
        return api
    
    extractor = twitter_setup()
    tweets = extractor.user_timeline(screen_name="Eskom_SA",
                                 count=200,
                                 include_rts=False)
                                     
    tweeter_data = pd.DataFrame(data=np.column_stack([[tweet.text for tweet in tweets],\
                                          [(tweet.created_at) for tweet in tweets]]),\
                                          columns=['Tweets','Date'])   
    return tweeter_data

# Function 2: Removing hashtags and the municipalities

Write a function which:
- Uses the function you wrote in the Analyse section to extract the hashtags and municipalities into it's own column in a new data frame. 

Function Specifications:
- The function should take in the pandas dataframe you created in Function 1 and return a new pandas dataframe. 

In [0]:
twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)

In [22]:
### START FUNCTION
def extract_municipality_hashtags(df):
    # your code here
    
    """Return a modified dataframe that includes two new columns with information
    about the municipality and hashtags of the tweet.
    
    Parameter
    - - - - -
    df: dataframe
        A pandas dataframe as the parameter.
    mun_dict : dictinary
        A dictionary used to store municipalities.
        
    Returns
    - - - - 
    df: dataframe
        An updated dataframe with two new columns; municipality and hashtags."""
    
    mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    # get the name of the municipality 
    def get_mun(a):
        for key in a.split():
            if key in  mun_dict.keys():
                return mun_dict[key]
        return np.nan
    
    # create a function that will extract the hashtags
    lst = []
    tweets = df['Tweets']
    for word in tweets:
        if '#' in word:
            lst.append(list(filter(lambda tweet:tweet.startswith('#'), word.lower().split())))
        else:
            lst.append('NaN')
        
    # apply the local function to the dataframe column containing the tweets
    df['Municipality'] = df['Tweets'].apply(get_mun)
    df['Hashtags'] = lst
    
    return df

# Function 3: Updating SQL Database with pyODBC

Write a function which:
- Connects and updates your SQL database. 

Function Specifications:
- The function should take in a pandas dataframe created in Function 2. 
- Connect to your SQL database.
- Update the table you store your tweets in.
- Not return any output.

In [ ]:
def pyodbc_twitter(connection, df, twitter_table):
    ### Code Here
    
    """Updates the SQL database with the new dataframe with two new columns;
    municipality and hashtags.
    
    Parameter
    - - - - -
    df: dataframe
        A pandas dataframe as the parameter.
    a: list
        A list of the dataframe columns with the records.
        
    Returns
    - - - - 
    Not return any output."""
    
    for row,column in df.iterrows():
        a = list(column)
        cur.execute("""INSERT INTO {}(Tweets_Date,Municipality,Hashtags,Tweets)\
                    VALUES(?,?,?,?)""".format(twitter_table), a[1],str(a[2]),str(a[3]),a[0]))
    connection.commit()
    connection.close()
    return None